### Initialization
![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)

In [27]:
import time
import json
import pprint
import config
import base64
import spotipy
import requests
import datetime
import pandas as pd
from functools import lru_cache
from bs4 import BeautifulSoup
from urllib.parse import urlencode
from spotipy.oauth2 import SpotifyClientCredentials

pd.set_option('display.max_columns', None)

In [28]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.clientId,
                                                           client_secret= config.clientSecret))

In [29]:
def get_playlist_tracks(username, playlist_id):
    results = sp.user_playlist_tracks(username, playlist_id)
    tracks = results["items"]
    while results["next"]:
        results = sp.next(results)
        tracks.extend(results["items"])
    return tracks

#### Functions

In [30]:
# selective call to function
class dataProfiling:

    def moreInfo(self, data):    
        # more data info
        print(f"MORE DATA INFO :")
        print("-" * 109)

        print(f"Data shape : {data.shape}\n")

        # counts no. total values in the data
        print(f"No. of values in the dataset : {data.size:,}")

        # counts no. of rows
        print(f"Total rows in the dataset : {len(data):,}")

        # counts no. of columns
        print(f"Total columns in the dataset : {len(data.columns):,}")

        # counts no. of missing values
        print(f"\nTotal null values : {data.isnull().sum().sum():,}")

        # counts no. of duplicates
        countDuplicates = data.duplicated().sum()
        print(f"Total duplicated rows : {format(countDuplicates, ',')}")

        # compute missing/duplicates ratio
        print(f"\nRATIO OF MISSING AND DUPLICATED VALUES IN OUR DATA :")
        print("-" * 109)

        # ratio for missing values
        isnull_ratio = (data.isnull().sum().sum() / data.size) * 100

        # ratio for duplicated values
        dup_ratio = (countDuplicates / len(data)) * 100

        # return results
        print(f"\nPercentage of null values in the data : {round(isnull_ratio, 2)}%")
        print(f"Percentage of duplicates in the data : {round(dup_ratio, 2)}%")
        print("\n")

    def valueCounts(self, data):
        # count no. of values per column
        print(f"\nCOUNT VALUES FOR EACH COLUMN :")
        print("-" * 109)
        for column in data.columns:
            countValues = data[column].value_counts(dropna=False).to_frame()
            totalValues = data[column].count()
            print(f"VALUE_COUNTS for column '{column}' : {len(countValues)} rows, {totalValues} values in Total\n")
            display(countValues)
            print("-" * 109)

    def nullColumns(self, data):
        # displays no. of missing values per column
        print(f"\nTOTAL MISSING VALUES FOR EACH COLUMN :")
        display(data.isnull().sum().to_frame().rename(columns={0: "MISSING VALUES"}))
        print("\n")

    def nullGroupby(self, data, column):
        # displays no. of missing values per category
        print(f"\nTOTAL MISSING VALUES PER COLUMN FOR EACH UNIQUE VALUE :")        
        display(data.isna().groupby(data[column]).sum())

        print("\n")
    def visualizeNulls(self, data):
        # visualize missing values
        print(f"VISAULIZE MISSNG VALUES :")
        print("-" * 109)

        # method 1
        print("METHOD 1")
        sns.heatmap(data.isnull().T,
                    xticklabels=False,
                    cmap="viridis",
                    cbar_kws={"label": "Missing Values"})

        plt.tight_layout()
        plt.show()
        print("\n")

        # method 2
        print("METHOD 2")
        plt.imshow(data.isna(), aspect="auto", interpolation="nearest", cmap="gray")
        plt.xlabel("Column Number")
        plt.ylabel("Sample Number");
        print("\n")

    def nullFeature(self, data, column):
        # displays rows where missing values are found of a specific column 
        print(f"DISPLAYS ROWS WHERE MISSING VALUES ARE FOUND OF A SPECIFIC FEATURE:")
        display(data[data[column].isna()])
        print("\n")

    def nullData(self, data):
        # displays rows where missing values are found
        print(f"DISPLAYS ROWS WHERE MISSING VALUES ARE FOUND :")
        display(data[data.isnull().any(axis=1)])
        print("\n")

    def allStats(self, data):
        # overall descerptive analysis (nuemrical and categorical)
        print(f"FULL DATA DESCRIPTIVE STATISTICS :")
        print("-" * 109)
        display(data.describe(include="all"))
        print("\n")

    def descriptiveData(self, data):
        # do quick descriptive statistics
        print(f"QUICK DESCRIPTIVE ANALYSIS :")
        print("-" * 109)
        display(data.describe())
        print("\n")

    def nonNumericStats(self, data):
        # categorical statistics
        print(f"NON-NUMERICAL STATISTICS :")
        print("-" * 109)
        display(data.describe(exclude="number"))
        print("\n")

    def uniqueColumns(self, data):
        # view all unique values for each column
        print(f"UNIQUE VALUES FOR EACH COLUMN :")
        print("-" * 109)

        for column in data.columns:
            uniqueValues = data[column].unique()
            uniqueCount = data[column].nunique()
            print(f"UNIQUE VALUES for column '{column}' : {len(uniqueValues)} including NaN values, {uniqueCount} excluding Nan values\n")
            print(f"{uniqueValues}\n")
            print("-" * 109)

    def plotUniques(self, data):
        # for each numerical feature compute number of unique entries
        unique_values = data.select_dtypes(include="number").nunique().sort_values()

        # plot information with y-axis in log-scale
        unique_values.plot.bar(logy=False, title="No. of unique values per feature", figsize=(25,7))
        plt.xticks(rotation=0)
        plt.tight_layout()
        plt.show()

# instance of the dataProfiling class
profiler = dataProfiling()

## experimental : *ground zero*

In [31]:
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials
# 
# lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'
# results = sp.artist_top_tracks(lz_uri)
# 
# for track in results['tracks'][:10]:
#     print('track    : ' + track['name'])
#     print('audio    : ' + track['preview_url'])
#     print('cover art: ' + track['album']['images'][0]['url'])
#     print()

In [32]:
# playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"
# playlist_URI = playlist_link.split("/")[-1].split("?")[0]
# track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [33]:
# # extracting tracks' uri from the playlist
# topGlobal_uris = [x["track"]["uri"] for x in sp.playlist_tracks(topGlobal)["items"]]
# pprint.pprint(topGlobal_uris)
# 
# # no. of tracks in the playlist
# print(len(topGlobal_uris))

In [34]:
# uri = 'spotify:artist:2WX2uTcsvV5OnS0inACecP'
# 
# results = sp.artist_albums(uri, album_type='album')
# albums = results['items']
# 
# while results['next']:
#     results = sp.next(results)
#     albums.extend(results['items'])
# 
# for album in albums:
#     print(album['name'])

In [35]:
# for song in sp.playlist_tracks(topGlobal)["items"]:
#     track_uri = song["track"]["uri"]
#     results = sp.search(q=track_uri, market="US")
#     pprint.pprint(results)

In [36]:
# playlistTracks = get_playlist_tracks("spotify", topGlobal)
# pprint.pprint(playlistTracks)

In [37]:
# j = sp.playlist_tracks(topGlobal)["items"]
# for song in j:
#     pprint.pprint(song)

In [38]:
# # hold track details
# track_details = []

# # get first page of playlist tracks
# results = sp.playlist_tracks(topGlobal)

# # While there are more pages (tracks) to be fetched
# while results:

#     # extract track info from the current page of the playlist
#     for song in results["items"]:
        
#         track_dict = {}

#         # TRACK
#         track_dict["track_uri"] = song["track"]["uri"]
#         track_dict["track_id"] = song["track"]["id"]
#         track_dict["track_name"] = song["track"]["name"]
#         track_dict["track_pop"] = song["track"]["popularity"]
#         track_dict["track_duration"] = song["track"]["duration_ms"]
#         track_dict["track_number"] = song["track"]["disc_number"]
#         track_dict["is_explicit"] = song["track"]["explicit"]

#         # ARTISTS
#         # list to hold information for all artists of the track
#         artist_ids = []
#         artist_names = []
#         artist_uris = []
#         artist_pops = []
#         artist_genres = []

#         for artist in song["track"]["artists"]:
#             artist_ids.append(artist["id"])
#             artist_names.append(artist["name"])
#             artist_uris.append(artist["uri"])
#             artist_info = sp.artist(artist["uri"])
#             artist_pops.append(artist_info["popularity"])
#             artist_genres.append(artist_info["genres"])

#         track_dict["artist_ids"] = artist_ids
#         track_dict["artist_names"] = artist_names
#         track_dict["artist_uris"] = artist_uris
#         track_dict["artist_pops"] = artist_pops
#         track_dict["artist_genres"] = artist_genres

#         # ALBUM
#         track_dict["album_name"] = song["track"]["album"]["name"]
#         track_dict["album_id"] = song["track"]["album"]["id"]
#         track_dict["album_type"] = song["track"]["album"]["album_type"]
#         track_dict["album_release"] = song["track"]["album"]["release_date"]
#         track_dict["album_totalTracks"] = song["track"]["album"]["total_tracks"]
#         track_dict["album_cover"] = song["track"]["album"]["images"][0]["url"]

#         # AUDIO FEATURES
#         audio_features = sp.audio_features(track_dict["track_id"])[0]  # get the first (and only) item
#         track_dict.update(audio_features)  # add the audio features to the track dictionary

#         # append the track_dict to the list
#         track_details.append(track_dict)

#     # get next page of tracks
#     results = sp.next(results)

# # convert the list of dictionaries to a DataFrame
# x = pd.DataFrame(track_details)
# x

In [39]:
# j = sp.playlist_tracks(topGlobal)["items"]
# for song in j:
#     pprint.pprint(song)

In [40]:
# for playlist_name, playlist_id in playlists.items():
#     print(f"Genres for playlist: {playlist_name}")
#     # Fetching playlist data
#     playlist = sp.playlist(playlist_id)
#     # Fetching tracks in the playlist
#     tracks = playlist['tracks']['items']
#     for track in tracks:
#         if track['track']['artists']:
#             # Fetching artist data for each track
#             artist = sp.artist(track['track']['artists'][0]['uri'])
#             # The genres associated with the artist
#             genres = artist['genres']
#             print(f"Track: {track['track']['name']}, Artist: {track['track']['artists'][0]['name']}, Genres: {genres}")
#         else:
#             print(f"Track: {track['track']['name']} has no associated artist.")
#     print("\n")

In [41]:
# topGlobal = "37i9dQZEVXbNG2KDcFcKOF"
# playlist = sp.user_playlist_tracks("spotify", topGlobal)
# pprint.pprint(playlist)
  
# print(list(playlist.keys())) # Let's look at items and total:
# print("Total number of songs in the playlist: ",playlist["total"]) #  Let's check items:

In [42]:
# topGlobal = "37i9dQZEVXbNG2KDcFcKOF"
# playlist = sp.user_playlist_tracks("spotify", topGlobal)
# pprint.pprint(playlist)
  
# print(list(playlist.keys())) # Let's look at items and total:
# print("Total number of songs in the playlist: ",playlist["total"]) #  Let's check items:

In [43]:
# pprint.pprint(sp.search(q="O Stella"))

In [44]:
# # Artist's URI
# artist_uri = "5YGY8feqx7naU7z4HrwZM6"

# # Fetching artist data
# artist = sp.artist(artist_uri)

# # The genres associated with the artist
# genres = artist['genres']

# print(genres)

In [45]:
# topGlobal = "37i9dQZEVXbNG2KDcFcKOF"
# playlist = sp.user_playlist_tracks("spotify", topGlobal)
# # pprint.pprint(playlist)

# # list to hold track details
# track_details = []

# # first page of playlist tracks
# results = sp.playlist_tracks(topGlobal)

# # While there are more pages (tracks) to be fetched
# while results:

#     # extract track info from the current page of the playlist
#     for song in results["items"]:
        
#         track_dict = {}

#         # TRACK
#         track_dict["track_uri"] = song["track"]["uri"]
#         track_dict["track_id"] = song["track"]["id"]
#         track_dict["track_name"] = song["track"]["name"]
#         track_dict["track_pop"] = song["track"]["popularity"]
#         track_dict["track_duration"] = song["track"]["duration_ms"]
#         track_dict["track_number"] = song["track"]["disc_number"]
#         track_dict["is_explicit"] = song["track"]["explicit"]

#         # ARTIST
#         artist_uri = song["track"]["artists"][0]["uri"]
#         artist_info = sp.artist(artist_uri)
#         track_dict["artist_id"] = song["track"]["artists"][0]["id"]
#         track_dict["artist_name"] = song["track"]["artists"][0]["name"]
#         track_dict["artist_pop"] = artist_info["popularity"]
#         track_dict["artist_genres"] = artist_info["genres"]

#         # ALBUM
#         track_dict["album_name"] = song["track"]["album"]["name"]
#         track_dict["album_id"] = song["track"]["album"]["id"]
#         track_dict["album_type"] = song["track"]["album"]["album_type"]
#         track_dict["album_release"] = song["track"]["album"]["release_date"]
#         track_dict["album_totalTracks"] = song["track"]["album"]["total_tracks"]
#         track_dict["album_cover"] = song["track"]["album"]["images"][0]["url"]

#         # AUDIO FEATURES
#         audio_features = sp.audio_features(track_dict["track_id"])[0]  # get the first (and only) item
#         track_dict.update(audio_features)  # add the audio features to the track dictionary

#         # append the track_dict to the list
#         track_details.append(track_dict)

#     # get next page of tracks
#     results = sp.next(results)

# # convert the list of dictionaries to a DataFrame
# spotipyData = pd.DataFrame(track_details)
# spotipyData

In [46]:
# spotipyData["artist_genres"].loc[spotipyData["artist_name"] == "Yandel"]

In [47]:
# @lru_cache(maxsize=1000)
# def get_artist_info(artist_uri):
#     return sp.artist(artist_uri)

# @lru_cache(maxsize=1000)
# def get_audio_features(track_id):
#     return sp.audio_features(track_id)[0]

# # maximum number of retries for the API call
# max_retries = 5

# # list to hold track details
# track_details = []

# # iterate over each playlist in the dictionary
# for playlist_name, playlist_id in playlists.items():

#     for i in range(max_retries):
#         try:
#             # first page of playlist tracks
#             results = sp.playlist_tracks(playlist_id)

#             # While there are more pages (tracks) to be fetched
#             while results:

#                 # extract track info from the current page of the playlist
#                 for song in results["items"]:
                    
#                     track_dict = {}

#                     # add the playlist name
#                     track_dict["playlist_name"] = playlist_name

#                     # TRACK
#                     track_dict["track_uri"] = song["track"]["uri"]
#                     track_dict["track_id"] = song["track"]["id"]
#                     track_dict["track_name"] = song["track"]["name"]
#                     track_dict["track_pop"] = song["track"]["popularity"]
#                     track_dict["track_duration"] = song["track"]["duration_ms"]
#                     track_dict["track_number"] = song["track"]["disc_number"]
#                     track_dict["is_explicit"] = song["track"]["explicit"]

#                     # ARTIST
#                     artist_uri = song["track"]["artists"][0]["uri"]
#                     track_dict["artist_id"] = song["track"]["artists"][0]["id"]
#                     track_dict["artist_name"] = song["track"]["artists"][0]["name"]
                    
#                     artist_info = get_artist_info(artist_uri)
#                     track_dict["artist_pop"] = artist_info["popularity"]
#                     track_dict["artist_genres"] = artist_info["genres"]

#                     # ALBUM
#                     track_dict["album_name"] = song["track"]["album"]["name"]
#                     track_dict["album_id"] = song["track"]["album"]["id"]
#                     track_dict["album_type"] = song["track"]["album"]["album_type"]
#                     track_dict["album_release"] = song["track"]["album"]["release_date"]
#                     track_dict["album_totalTracks"] = song["track"]["album"]["total_tracks"]
#                     track_dict["album_cover"] = song["track"]["album"]["images"][0]["url"]

#                     # AUDIO FEATURES
#                     audio_features = get_audio_features(track_dict["track_id"])  # get the first (and only) item
#                     track_dict.update(audio_features)  # add the audio features to the track dictionary

#                     # append the track_dict to the list
#                     track_details.append(track_dict)

#                 # get next page of tracks
#                 results = sp.next(results)

#             # break loop if the API call was successful
#             break

#         except spotipy.exceptions.SpotifyException as e:
#             # if exception is a rate limiting error (error 429), wait and retry
#             if e.http_status == 429:
#                 print("Rate limit exceeded. Waiting...")
#                 time.sleep(int(e.headers.get('Retry-After', 10)))
#             else:
#                 # if different kind of error, we will handle it differently, let's see..
#                 # or re-raise the exception if it's not something your script can recover from
#                 raise e

# # convert the list of dictionaries to a DataFrame
# spotipyData = pd.DataFrame(track_details)
# spotipyData

In [48]:
# @lru_cache(maxsize=1000)
# def get_artist_info(artist_uri):
#     return sp.artist(artist_uri)

# @lru_cache(maxsize=1000)
# def get_audio_features(track_id):
#     return sp.audio_features(track_id)[0]

# # maximum number of retries for the API call
# max_retries = 5

# # list to hold track details
# track_details = []

# # iterate over each playlist in the dictionary
# for playlist_name, playlist_id in playlists.items():

#     for i in range(max_retries):
#         try:
#             # first page of playlist tracks
#             results = sp.playlist_tracks(playlist_id)

#             # While there are more pages (tracks) to be fetched
#             while results:

#                 # extract track info from the current page of the playlist
#                 for song in results["items"]:
                    
#                     track_dict = {}

#                     # add the playlist name
#                     track_dict["playlist_name"] = playlist_name

#                     # TRACK
#                     track_dict["track_uri"] = song["track"]["uri"]
#                     track_dict["track_id"] = song["track"]["id"]
#                     track_dict["track_name"] = song["track"]["name"]
#                     track_dict["track_pop"] = song["track"]["popularity"]
#                     track_dict["track_duration"] = song["track"]["duration_ms"]
#                     track_dict["track_number"] = song["track"]["disc_number"]
#                     track_dict["is_explicit"] = song["track"]["explicit"]

#                     # ARTIST
#                     artist_uri = song["track"]["artists"][0]["uri"]
#                     track_dict["artist_id"] = song["track"]["artists"][0]["id"]
#                     track_dict["artist_name"] = song["track"]["artists"][0]["name"]
                    
#                     artist_info = get_artist_info(artist_uri)
#                     track_dict["artist_pop"] = artist_info["popularity"]
#                     track_dict["artist_genres"] = artist_info["genres"]

#                     # ALBUM
#                     track_dict["album_name"] = song["track"]["album"]["name"]
#                     track_dict["album_id"] = song["track"]["album"]["id"]
#                     track_dict["album_type"] = song["track"]["album"]["album_type"]
#                     track_dict["album_release"] = song["track"]["album"]["release_date"]
#                     track_dict["album_totalTracks"] = song["track"]["album"]["total_tracks"]
#                     track_dict["album_cover"] = song["track"]["album"]["images"][0]["url"]

#                     # AUDIO FEATURES
#                     audio_features = get_audio_features(track_dict["track_id"])  # get the first (and only) item
#                     track_dict.update(audio_features)  # add the audio features to the track dictionary

#                     # append the track_dict to the list
#                     track_details.append(track_dict)

#                 # get next page of tracks
#                 results = sp.next(results)

#             # break loop if the API call was successful
#             break

#         except spotipy.exceptions.SpotifyException as e:
#             # if exception is a rate limiting error (error 429), wait and retry
#             if e.http_status == 429:
#                 print("Rate limit exceeded. Waiting...")
#                 time.sleep(int(e.headers.get('Retry-After', 10)))
#             else:
#                 # if different kind of error, we will handle it differently, let's see..
#                 # or re-raise the exception if it's not something your script can recover from
#                 raise e

# # convert the list of dictionaries to a DataFrame
# spotipyData = pd.DataFrame(track_details)
# spotipyData

In [49]:
# # list to hold track details
# track_details = []

# # iterate over each playlist in the dictionary
# for playlist_name, playlist_id in playlists.items():

#     # first page of playlist tracks
#     results = sp.playlist_tracks(playlist_id)

#     # While there are more pages (tracks) to be fetched
#     while results:

#         # extract track info from the current page of the playlist
#         for song in results["items"]:
            
#             track_dict = {}

#             # add the playlist name
#             track_dict["playlist_name"] = playlist_name

#             # TRACK
#             track_dict["trackURI"] = song["track"]["uri"]
#             track_dict["trackID"] = song["track"]["id"]
#             track_dict["trackName"] = song["track"]["name"]
#             track_dict["trackPopularity"] = song["track"]["popularity"]
#             track_dict["trackDuration"] = song["track"]["duration_ms"]
#             track_dict["trackNumber"] = song["track"]["disc_number"]
#             track_dict["isExplicit"] = song["track"]["explicit"]

#             # ARTIST
#             artist_uri = song["track"]["artists"][0]["uri"]
#             track_dict["artistID"] = song["track"]["artists"][0]["id"]
#             track_dict["artistName"] = song["track"]["artists"][0]["name"]
#             artist_info = sp.artist(artist_uri)
#             track_dict["asrtistPopularity"] = artist_info["popularity"]
#             track_dict["artistGenre"] = artist_info["genres"]

#             # ALBUM
#             track_dict["albumName"] = song["track"]["album"]["name"]
#             track_dict["albumID"] = song["track"]["album"]["id"]
#             track_dict["albumType"] = song["track"]["album"]["album_type"]
#             track_dict["releaseDate"] = song["track"]["album"]["release_date"]
#             track_dict["album_numberTracks"] = song["track"]["album"]["total_tracks"]
#             track_dict["albumCover"] = song["track"]["album"]["images"][0]["url"]

#             # AUDIO FEATURES
#             audio_features = sp.audio_features(track_dict["trackID"])[0]  # get the first (and only) item
#             track_dict.update(audio_features)  # add the audio features to the track dictionary

#             # append the track_dict to the list
#             track_details.append(track_dict)

#         # get next page of tracks
#         results = sp.next(results)

# # convert the list of dictionaries to a DataFrame
# spotipyData = pd.DataFrame(track_details)
# spotipyData

In [50]:
# # list to hold track details
# track_details = []

# # iterate over each playlist in the dictionary
# for playlist_name, playlist_id in playlists.items():

#     # first page of playlist tracks
#     results = sp.playlist_tracks(playlist_id)

#     # While there are more pages (tracks) to be fetched
#     while results:

#         # extract track info from the current page of the playlist
#         for song in results["items"]:
            
#             track_dict = {}

#             # add the playlist name
#             track_dict["playlist_name"] = playlist_name

#             # TRACK
#             track_dict["trackURI"] = song["track"]["uri"]

#             # ARTIST
#             artist_uri = song["track"]["artists"][0]["uri"]
#             if isinstance(artist_uri, str):  # check if artist_uri is a string
#                 track_dict["artist_id"] = song["track"]["artists"][0]["id"]
#                 track_dict["artist_name"] = song["track"]["artists"][0]["name"]

#                 artist_info = get_artist_info(artist_uri)  # call the function if artist_uri is a string
                
#                 track_dict["artist_pop"] = artist_info["popularity"]
#                 track_dict["artist_genres"] = artist_info["genres"]
#             else:
#                 print(f"Unexpected type for artist_uri: {type(artist_uri)}")

#             # append the track_dict to the list
#             track_details.append(track_dict)

#         # get next page of tracks
#         results = sp.next(results)

# # convert the list of dictionaries to a DataFrame
# spotipyData = pd.DataFrame(track_details)
# spotipyData

## data preparation

In [51]:
playlists ={
    "Who's That Girl": "0cLPcrlTcogy15QDfsQMWH",
    "Last Century 50s-80s": "0hGSCdsqOASoxNE9HIF8Qv",
    "Eurovision 2023": "37i9dQZF1DWVCKO3xAlT1Q",
    "Soft 50s": "37i9dQZF1DWXcg95telZlE",
    "50s Party": "37i9dQZF1DWSwFS0Z6E1ep",
    "All Out 50s": "37i9dQZF1DWSV3Tk4GO2fq",
    "All Out 60s": "37i9dQZF1DXaKIA8E7WcJj",
    "Années 60": "37i9dQZF1DX7Uol5MpckMS",
    "All Out 70s": "37i9dQZF1DWTJ7xPn4vNaz",
    "Best of Rock 1970s": "37i9dQZF1DXe9Gx5fVy1RT",
    "Top Hits of 1970s": "37i9dQZF1DWXQyLTHGuTIz",
    "topGlobal": "37i9dQZEVXbNG2KDcFcKOF"}

In [52]:
@lru_cache(maxsize=1000)
def get_artist_info(artist_uri):
    return sp.artist(artist_uri)

@lru_cache(maxsize=1000)
def get_audio_features(track_id):
    return sp.audio_features(track_id)[0]

# maximum number of retries for the API call
max_retries = 5

# list to hold track details
track_details = []

# iterate over each playlist in the dictionary
for playlist_name, playlist_id in playlists.items():

    for i in range(max_retries):
        try:
            # first page of playlist tracks
            results = sp.playlist_tracks(playlist_id)

            # While there are more pages (tracks) to be fetched
            while results:

                # extract track info from the current page of the playlist
                for song in results["items"]:
                    
                    track_dict = {}

                    # add the playlist name
                    track_dict["fromPlaylist"] = playlist_name

                    # TRACK
                    # track_dict["trackURI"] = song["track"]["uri"]
                    # track_dict["trackID"] = song["track"]["id"]
                    track_dict["trackName"] = song["track"]["name"]
                    track_dict["trackPopularity"] = song["track"]["popularity"]
                    # track_dict["trackDuration"] = song["track"]["duration_ms"]
                    track_dict["trackNumber"] = song["track"]["disc_number"]
                    track_dict["isExplicit"] = song["track"]["explicit"]

                    # ARTIST
                    artist_uri = song["track"]["artists"][0]["uri"]
                    if isinstance(artist_uri, str):  # check if artist_uri is a string
                        track_dict["artistID"] = song["track"]["artists"][0]["id"]
                        track_dict["artistName"] = song["track"]["artists"][0]["name"]
        
                        artist_info = get_artist_info(artist_uri)  # call the function if artist_uri is a string
                        
                        track_dict["artistPop"] = artist_info["popularity"]
                        track_dict["artistGenre"] = artist_info["genres"][0] if artist_info["genres"] else None

                    else:
                        print(f"Unexpected type for artist_uri: {type(artist_uri)}")

                    # ALBUM
                    track_dict["albumName"] = song["track"]["album"]["name"]
                    track_dict["albumID"] = song["track"]["album"]["id"]
                    track_dict["albumType"] = song["track"]["album"]["album_type"]
                    release_date = song["track"]["album"]["release_date"]
                    if release_date is not None:
                        track_dict["releaseDate"] = pd.to_datetime(release_date).year
                    else:
                        track_dict["releaseDate"] = None
                    track_dict["album_nTracks"] = song["track"]["album"].get("total_tracks", None)
                    album_images = song["track"]["album"]["images"]
                    if album_images:
                        track_dict["albumCover"] = album_images[0]["url"]
                    else:
                        track_dict["albumCover"] = None

                    # AUDIO FEATURES
                    track_id = song["track"]["id"] # track_dict["trackID"]
                    if track_id is not None:
                        audio_features = get_audio_features(track_id)
                        track_dict.update(audio_features)
                    else:
                        print(f"Unexpected type for trackID: {type(track_id)}")

                    # append the track_dict to the list
                    track_details.append(track_dict)

                # get next page of tracks
                results = sp.next(results)

            # break loop if the API call was successful
            break

        except spotipy.exceptions.SpotifyException as e:
            # if exception is a rate limiting error (error 429), wait and retry
            if e.http_status == 429:
                print("Rate limit exceeded. Waiting...")
                time.sleep(int(e.headers.get("Retry-After", 10)))
            else:
                # if different kind of error, we will handle it differently, let's see..
                # or re-raise the exception if it's not something your script can recover from
                raise e

# convert the list of dictionaries to a DataFrame
spotipyData = pd.DataFrame(track_details)
spotipyData

Unexpected type for artist_uri: <class 'NoneType'>
Unexpected type for trackID: <class 'NoneType'>
Unexpected type for artist_uri: <class 'NoneType'>
Unexpected type for trackID: <class 'NoneType'>
Unexpected type for artist_uri: <class 'NoneType'>
Unexpected type for trackID: <class 'NoneType'>
Unexpected type for artist_uri: <class 'NoneType'>
Unexpected type for trackID: <class 'NoneType'>


,fromPlaylist,trackName,trackPopularity,trackNumber,isExplicit,artistID,artistName,artistPop,artistGenre,albumName,albumID,albumType,releaseDate,album_nTracks,albumCover,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Who's That Girl,Violet,64,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.287,0.780,7.0,-6.493,1.0,0.0469,0.04900,0.000037,0.1840,0.3830,167.278,audio_features,6CHENx8iqzMwavTIz8s0gm,spotify:track:6CHENx8iqzMwavTIz8s0gm,https://api.spotify.com/v1/tracks/6CHENx8iqzMw...,https://api.spotify.com/v1/audio-analysis/6CHE...,204933.0,4.0
1,Who's That Girl,Miss World,54,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.331,0.722,7.0,-8.461,1.0,0.0408,0.00204,0.000053,0.2560,0.3490,125.575,audio_features,3yMFBuIdPBdJkkzaPBDjKY,spotify:track:3yMFBuIdPBdJkkzaPBDjKY,https://api.spotify.com/v1/tracks/3yMFBuIdPBdJ...,https://api.spotify.com/v1/audio-analysis/3yMF...,180067.0,4.0
2,Who's That Girl,Doll Parts,61,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.245,0.542,7.0,-7.159,1.0,0.0324,0.14700,0.000004,0.1020,0.3650,204.824,audio_features,49t1GWE6ZiEoBgN92oMDdM,spotify:track:49t1GWE6ZiEoBgN92oMDdM,https://api.spotify.com/v1/tracks/49t1GWE6ZiEo...,https://api.spotify.com/v1/audio-analysis/49t1...,211960.0,4.0
3,Who's That Girl,Rock Star,47,1,True,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.308,0.868,4.0,-6.440,1.0,0.1440,0.00514,0.000045,0.2700,0.2500,125.782,audio_features,72xpgWWuEyLWFFTIKnrAbZ,spotify:track:72xpgWWuEyLWFFTIKnrAbZ,https://api.spotify.com/v1/tracks/72xpgWWuEyLW...,https://api.spotify.com/v1/audio-analysis/72xp...,162200.0,4.0
4,Who's That Girl,Lark,0,1,False,6mKqFxGMS5TGDZI3XkT5Rt,Angel Olsen,55.0,alternative americana,All Mirrors,0RedX0LZkGUFoRwFntAaI0,album,2019.0,11.0,https://i.scdn.co/image/ab67616d0000b2732b31fc...,0.447,0.560,7.0,-8.360,1.0,0.0316,0.00311,0.099100,0.2200,0.0422,98.986,audio_features,13rEyddPoVpxXWWv2aTjqo,spotify:track:13rEyddPoVpxXWWv2aTjqo,https://api.spotify.com/v1/tracks/13rEyddPoVpx...,https://api.spotify.com/v1/audio-analysis/13rE...,378827.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,topGlobal,Cruel Summer,91,1,False,06HL4z0CvFAxyc27GXpf02,Taylor Swift,100.0,pop,Lover,1NAmidJlEaVgA3MpcPFYGq,album,2019.0,18.0,https://i.scdn.co/image/ab67616d0000b273e787cf...,0.552,0.702,9.0,-5.707,1.0,0.1570,0.11700,0.000021,0.1050,0.5640,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427.0,4.0
1611,topGlobal,Coco Chanel,92,1,True,5XJDexmWFLWOkjOEjOVX3e,Eladio Carrion,86.0,trap boricua,3MEN2 KBRN,6BGN5CVd7koJApotl5Bj8u,album,2023.0,18.0,https://i.scdn.co/image/ab67616d0000b273fc8563...,0.683,0.764,2.0,-5.995,1.0,0.0441,0.04270,0.000000,0.0963,0.1370,149.949,audio_features,0PB0O24JqAuNdOAFVJljMS,spotify:track:0PB0O24JqAuNdOAFVJljMS,https://api.spotify.com/v1/tracks/0PB0O24JqAuN...,https://api.spotify.com/v1/audio-analysis/0PB0...,160325.0,4.0
1612,topGlobal,Chemical,90,1,True,246dkjvS1zLTtiykXe5h60,Post Malone,89.0,dfw rap,Chemical,7qcSUc5Af63mhfTF60KTEA,single,2023.0,1.0,https://i.scdn.co/image/ab67616d0000b273f76f8d...,0.498,0.897,2.0,-4.907,1.0,0.0450,0.00136,0.000000,0.1180,0.3730,169.902,audio_features,5w40ZYhbBMAlHYNDaVJIUu,spotify:track:5w40ZYhbBMAlHYNDaVJIUu,https:/

## data profiling

### to work on later

In [53]:
# indexes_to_delete = [501, 1287, 1410, 832, 676, 992, 1007, 1041, 771, 775, 1243, 798, 962, 977, 934, 753, 779, 1219, 611, 567, 765, 758, 1502, 928, 1023, 1025, 582, 769, 1245,
#                      822, 1038, 1479, 980, 881, 974, 1009, 1477, 981, 542, 1480, 935, 1206, 1037]
# data = data.drop(indexes_to_delete).copy()

In [54]:
# data[["fromPlaylist", "trackID", "trackName", "releaseDate"]].loc[data.duplicated(["trackID", "trackName"], keep=False)].sort_values(by="trackID").head(50)

In [55]:
# data[data.duplicated(["trackID", "trackName"], keep=False)].count()

### as for now..

```python
# use these methods to analyze the data
profiler.moreInfo(data)
profiler.valueCounts(data)
profiler.nullColumns(data)
profiler.nullGroupby(data, column)
profiler.visualizeNulls(data)
profiler.nullFeature(data, column)
profiler.nullData(data)
profiler.descriptiveData(data)
profiler.nonNumericStats(data)
profiler.uniqueColumns(data)
profiler.plotUniques(data)
```

In [56]:
databackup = spotipyData.copy()

In [57]:
profiler.nullColumns(spotipyData)


TOTAL MISSING VALUES FOR EACH COLUMN :


,MISSING VALUES
fromPlaylist,0
trackName,0
trackPopularity,0
trackNumber,0
isExplicit,0
artistID,4
artistName,4
artistPop,4
artistGenre,55
albumName,0


In [58]:
profiler.nullData(spotipyData)

DISPLAYS ROWS WHERE MISSING VALUES ARE FOUND :


,fromPlaylist,trackName,trackPopularity,trackNumber,isExplicit,artistID,artistName,artistPop,artistGenre,albumName,albumID,albumType,releaseDate,album_nTracks,albumCover,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
61,Who's That Girl,Serpent,0,1,False,5ptfrHC6idq4KnsXBk5tup,Tenci,25.0,None,My Heart Is An Open Field,1XJMmPYcTYg19hBGNZp0V1,album,2020.0,9.0,https://i.scdn.co/image/ab67616d0000b2737276ae...,0.227,0.373,2.0,-13.904,1.0,0.0347,0.60200,0.818000,0.0732,0.1420,171.620,audio_features,3JhTxHuoDSeZc9Kan24Vwo,spotify:track:3JhTxHuoDSeZc9Kan24Vwo,https://api.spotify.com/v1/tracks/3JhTxHuoDSeZ...,https://api.spotify.com/v1/audio-analysis/3JhT...,312000.0,3.0
62,Who's That Girl,Joy,0,1,False,5ptfrHC6idq4KnsXBk5tup,Tenci,25.0,None,My Heart Is An Open Field,1XJMmPYcTYg19hBGNZp0V1,album,2020.0,9.0,https://i.scdn.co/image/ab67616d0000b2737276ae...,0.308,0.318,2.0,-12.434,1.0,0.0425,0.89000,0.015400,0.1170,0.0673,196.904,audio_features,35Fv5bmCfIyPsuMLQZryf2,spotify:track:35Fv5bmCfIyPsuMLQZryf2,https://api.spotify.com/v1/tracks/35Fv5bmCfIyP...,https://api.spotify.com/v1/audio-analysis/35Fv...,252610.0,3.0
63,Who's That Girl,Joy 2,0,1,False,5ptfrHC6idq4KnsXBk5tup,Tenci,25.0,None,My Heart Is An Open Field,1XJMmPYcTYg19hBGNZp0V1,album,2020.0,9.0,https://i.scdn.co/image/ab67616d0000b2737276ae...,0.422,0.311,2.0,-11.324,1.0,0.0372,0.93900,0.024500,0.1110,0.1180,117.718,audio_features,2BgbL846XAAXqMvtBEgxC4,spotify:track:2BgbL846XAAXqMvtBEgxC4,https://api.spotify.com/v1/tracks/2BgbL846XAAX...,https://api.spotify.com/v1/audio-analysis/2Bgb...,301296.0,4.0
226,Who's That Girl,My Jamaican Guy,46,1,False,2f9ZiYA2ic1r1voObUimdd,Grace Jones,57.0,None,Living My Life,4EsGuoM79PDO7tQwqAwjKC,album,1982.0,7.0,https://i.scdn.co/image/ab67616d0000b273f6444f...,0.751,0.910,7.0,-10.321,1.0,0.0608,0.07870,0.037900,0.0706,0.4890,95.392,audio_features,7zcC5yZZp1IKnI73ztsIaE,spotify:track:7zcC5yZZp1IKnI73ztsIaE,https://api.spotify.com/v1/tracks/7zcC5yZZp1IK...,https://api.spotify.com/v1/audio-analysis/7zcC...,362293.0,4.0
227,Who's That Girl,Cry Now - Laugh Later,20,1,False,2f9ZiYA2ic1r1voObUimdd,Grace Jones,57.0,None,Living My Life,4EsGuoM79PDO7tQwqAwjKC,album,1982.0,7.0,https://i.scdn.co/image/ab67616d0000b273f6444f...,0.777,0.787,10.0,-11.401,0.0,0.0736,0.03610,0.022100,0.0970,0.6330,118.699,audio_features,7jhMYVobMCo4psxoo0oliJ,spotify:track:7jhMYVobMCo4psxoo0oliJ,https://api.spotify.com/v1/tracks/7jhMYVobMCo4...,https://api.spotify.com/v1/audio-analysis/7jhM...,303600.0,4.0
228,Who's That Girl,Unlimited Capacity For Love,28,1,False,2f9ZiYA2ic1r1voObUimdd,Grace Jones,57.0,None,Living My Life,4EsGuoM79PDO7tQwqAwjKC,album,1982.0,7.0,https://i.scdn.co/image/ab67616d0000b273f6444f...,0.779,0.646,0.0,-10.239,1.0,0.0306,0.14300,0.049200,0.0739,0.9630,115.330,audio_features,2Sh1Jf6hK1RcQjWQgq4t00,spotify:track:2Sh1Jf6hK1RcQjWQgq4t00,https://api.spotify.com/v1/tracks/2Sh1Jf6hK1Rc...,https://api.spotify.com/v1/audio-analysis/2Sh1...,346200.0,4.0
283,Who's That Girl,Dead Pursuits,18,1,False,6nbtlXRy0S6adYpDVoRdNi,Anna B Savage,34.0,None,A Common Turn,6RQArjhEoHV6Pvoc2kAtrw,album,2021.0,10.0,https://i.scdn.co/image/ab67616d0000b2731d9800...,0.340,0.320,8.0,-12.015,1.0,0.0298,0.63900,0.000282,0.1630,0.0558,107.317,audio_features,6woUuWuGeKMxBogELVcTMy,spotify:track:6woUuWuGeKMxBogELVcTMy,https://api.spotify.com/v1/tracks/6woUuWuGeKMx...,https://api.spotify.com/v1/audio-analysis/6woU...,259173.0,4.0
284,Who's That Girl,Two,16,1,False,6nbtlXRy0S6adYpDVoRdNi,Anna B Savage,34.0,None,A Common Turn,6RQArjhEoHV6Pvoc2kAtrw,album,2021.0,10.0,https://i.scdn.co/image/ab67616d0000b2731d9800...,0.413,0.473,7.0,-10.440,0.0,0.0477,0.06480,0.337000,0.1100,0.1100,132.489,audio_features,4kBPXjGAeKzDTwk9m6FqLx,spotify:track:4kBPXjGAeKzDTwk9m6FqLx,https://api.spotify.com/v1/tracks/4kBPXjGAeKzD...,https://api.spotify.com/v1/audio-analysis/4kBP...,309081.0,3.0
285,W

In [59]:
profiler.moreInfo(spotipyData)

MORE DATA INFO :
-------------------------------------------------------------------------------------------------------------
Data shape : (1615, 33)

No. of values in the dataset : 53,295
Total rows in the dataset : 1,615
Total columns in the dataset : 33

Total null values : 159
Total duplicated rows : 1

RATIO OF MISSING AND DUPLICATED VALUES IN OUR DATA :
-------------------------------------------------------------------------------------------------------------

Percentage of null values in the data : 0.3%
Percentage of duplicates in the data : 0.06%




In [61]:
spotipyData = spotipyData.dropna(subset=["danceability"])
spotipyData = spotipyData.drop_duplicates(subset="id")

In [62]:
profiler.nonNumericStats(spotipyData)

NON-NUMERICAL STATISTICS :
-------------------------------------------------------------------------------------------------------------


,fromPlaylist,trackName,isExplicit,artistID,artistName,artistGenre,albumName,albumID,albumType,albumCover,type,id,uri,track_href,analysis_url
count,1398,1398,1398,1398,1398,1349,1398,1398,1398,1398,1398,1398,1398,1398,1398
unique,12,1364,2,569,569,177,892,909,3,909,1,1398,1398,1398,1398
top,Who's That Girl,Venus,False,7w29UYBi0qsHi5RTcv3lmA,Björk,adult standards,Cuz I Love You (Super Deluxe),2KJjOBX280F3hZZE1xO33O,album,https://i.scdn.co/image/ab67616d0000b2737bebcd...,audio_features,6CHENx8iqzMwavTIz8s0gm,spotify:track:6CHENx8iqzMwavTIz8s0gm,https://api.spotify.com/v1/tracks/6CHENx8iqzMw...,https://api.spotify.com/v1/audio-analysis/6CHE...
freq,467,3,1343,50,50,141,8,8,1197,8,1398,1,1,1,1


In [63]:
spotipyData

,fromPlaylist,trackName,trackPopularity,trackNumber,isExplicit,artistID,artistName,artistPop,artistGenre,albumName,albumID,albumType,releaseDate,album_nTracks,albumCover,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Who's That Girl,Violet,64,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.287,0.780,7.0,-6.493,1.0,0.0469,0.04900,0.000037,0.1840,0.3830,167.278,audio_features,6CHENx8iqzMwavTIz8s0gm,spotify:track:6CHENx8iqzMwavTIz8s0gm,https://api.spotify.com/v1/tracks/6CHENx8iqzMw...,https://api.spotify.com/v1/audio-analysis/6CHE...,204933.0,4.0
1,Who's That Girl,Miss World,54,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.331,0.722,7.0,-8.461,1.0,0.0408,0.00204,0.000053,0.2560,0.3490,125.575,audio_features,3yMFBuIdPBdJkkzaPBDjKY,spotify:track:3yMFBuIdPBdJkkzaPBDjKY,https://api.spotify.com/v1/tracks/3yMFBuIdPBdJ...,https://api.spotify.com/v1/audio-analysis/3yMF...,180067.0,4.0
2,Who's That Girl,Doll Parts,61,1,False,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.245,0.542,7.0,-7.159,1.0,0.0324,0.14700,0.000004,0.1020,0.3650,204.824,audio_features,49t1GWE6ZiEoBgN92oMDdM,spotify:track:49t1GWE6ZiEoBgN92oMDdM,https://api.spotify.com/v1/tracks/49t1GWE6ZiEo...,https://api.spotify.com/v1/audio-analysis/49t1...,211960.0,4.0
3,Who's That Girl,Rock Star,47,1,True,5SHQUMAmEK5KmuSb0aDvsn,Hole,60.0,alternative rock,Live Through This,2Rwf2nPYZQ9aIe4QXACTC7,album,1994.0,12.0,https://i.scdn.co/image/ab67616d0000b273be245e...,0.308,0.868,4.0,-6.440,1.0,0.1440,0.00514,0.000045,0.2700,0.2500,125.782,audio_features,72xpgWWuEyLWFFTIKnrAbZ,spotify:track:72xpgWWuEyLWFFTIKnrAbZ,https://api.spotify.com/v1/tracks/72xpgWWuEyLW...,https://api.spotify.com/v1/audio-analysis/72xp...,162200.0,4.0
4,Who's That Girl,Lark,0,1,False,6mKqFxGMS5TGDZI3XkT5Rt,Angel Olsen,55.0,alternative americana,All Mirrors,0RedX0LZkGUFoRwFntAaI0,album,2019.0,11.0,https://i.scdn.co/image/ab67616d0000b2732b31fc...,0.447,0.560,7.0,-8.360,1.0,0.0316,0.00311,0.099100,0.2200,0.0422,98.986,audio_features,13rEyddPoVpxXWWv2aTjqo,spotify:track:13rEyddPoVpxXWWv2aTjqo,https://api.spotify.com/v1/tracks/13rEyddPoVpx...,https://api.spotify.com/v1/audio-analysis/13rE...,378827.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1610,topGlobal,Cruel Summer,91,1,False,06HL4z0CvFAxyc27GXpf02,Taylor Swift,100.0,pop,Lover,1NAmidJlEaVgA3MpcPFYGq,album,2019.0,18.0,https://i.scdn.co/image/ab67616d0000b273e787cf...,0.552,0.702,9.0,-5.707,1.0,0.1570,0.11700,0.000021,0.1050,0.5640,169.994,audio_features,1BxfuPKGuaTgP7aM0Bbdwr,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,https://api.spotify.com/v1/tracks/1BxfuPKGuaTg...,https://api.spotify.com/v1/audio-analysis/1Bxf...,178427.0,4.0
1611,topGlobal,Coco Chanel,92,1,True,5XJDexmWFLWOkjOEjOVX3e,Eladio Carrion,86.0,trap boricua,3MEN2 KBRN,6BGN5CVd7koJApotl5Bj8u,album,2023.0,18.0,https://i.scdn.co/image/ab67616d0000b273fc8563...,0.683,0.764,2.0,-5.995,1.0,0.0441,0.04270,0.000000,0.0963,0.1370,149.949,audio_features,0PB0O24JqAuNdOAFVJljMS,spotify:track:0PB0O24JqAuNdOAFVJljMS,https://api.spotify.com/v1/tracks/0PB0O24JqAuN...,https://api.spotify.com/v1/audio-analysis/0PB0...,160325.0,4.0
1612,topGlobal,Chemical,90,1,True,246dkjvS1zLTtiykXe5h60,Post Malone,89.0,dfw rap,Chemical,7qcSUc5Af63mhfTF60KTEA,single,2023.0,1.0,https://i.scdn.co/image/ab67616d0000b273f76f8d...,0.498,0.897,2.0,-4.907,1.0,0.0450,0.00136,0.000000,0.1180,0.3730,169.902,audio_features,5w40ZYhbBMAlHYNDaVJIUu,spotify:track:5w40ZYhbBMAlHYNDaVJIUu,https:/